In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

In [2]:
import parallelism
parallelism.set_max_num_tbb_threads(1)

 - [Compute equilibria of **twist-free** knots](#sec:twist_free)
 - [Compute equilibria of **constant-link** knots](#sec:constant_link)

----
<a id='sec:twist_free'></a>
# Compute the equilibium state of a **twist-free** knotted elastic rod

### Select a knot

In [23]:
# Load the centerline from file...
file = '../data/4_1-smooth.obj'
#file = '../data/L400-r0.2-UpTo9Crossings/4_1/0033.obj'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 1, 0.5, [rod_radius, rod_radius])
centerline = read_nodes_from_file(file)  # supported formats: obj, txt

pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])

In [13]:
# ...or generate a parametric knot
p, q, a, r = 2, 3, 2, 4
rod_radius = 0.2

material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = generate_curve(torus_knot(p=p, q=q, a=a, r=r), npts=102)
pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])

### Viewer

In [24]:
view = Viewer(rod_list, width=1024, height=640)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), …

### Equilibrium solve

In [25]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()
        
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer

In [26]:
# Optimize
report = elastic_knots.compute_equilibrium(
    rod_list, problemOptions, optimizerOptions, 
    fixedVars=fixedVars,
    externalForces=np.zeros(rod_list.numDoF()),
    softConstraints=[],
    callback=callback,
    hessianShift=hessianShift
)
view.update()

0	0.000702632	0.000154838	0.000154838	0.125	1
1	0.000686271	0.00018919	0.00018919	0.5	0
2	0.000668111	0.000199508	0.000199508	1	0
3	0.000640918	0.000157174	0.000157174	1	0
4	0.000633743	0.000268195	0.000268195	1	0
5	0.000596571	0.000143881	0.000143881	0.40625	0
6	0.00058546	0.000103241	0.000103241	0.375	0
7	0.000575426	6.50869e-05	6.50869e-05	0.0625	0
8	0.000573673	6.11693e-05	6.11693e-05	0.000976562	0
9	0.000573644	6.11096e-05	6.11096e-05	0.000244141	0
10	0.000573637	6.0926e-05	6.0926e-05	0.00390625	0
11	0.000573549	6.05617e-05	6.05617e-05	0.25	0
12	0.00056988	0.00287108	0.00287108	1	1
13	0.000568826	0.000705746	0.000705746	1	1
14	0.000568086	0.000175994	0.000175994	1	1
15	0.000567552	4.49186e-05	4.49186e-05	1	1
16	0.000567314	1.19217e-05	1.19217e-05	1	1
17	0.000567233	3.6257e-06	3.6257e-06	1	1
18	0.000567156	2.31249e-06	2.31249e-06	1	1
19	0.000567018	2.17088e-06	2.17088e-06	1	1
20	0.000566766	2.10522e-06	2.10522e-06	1	1
21	0.000566321	2.07746e-06	2.07746e-06	1	1
22	0.00056556	3.35066

### Save result to file

In [41]:
from helpers import write_obj
file = '../data/output.obj'
write_obj(file, rod_list)

### Modal analysis

In [42]:
from helpers import vibrational_analysis
fixedVars = []
problemOptions.projectContactHessianPSD = False
lambdas, modes = vibrational_analysis(rod_list, problemOptions, optimizerOptions, n_modes=10, fixedVars=fixedVars)

Computing vibrational modes
0	1.56342	8.36789e-11	8.36789e-11	0	0


In [43]:
# Visualize vibrational modes
import mode_viewer
ndof = rod_list.numDoF()
amplitude = pr.restLength() / 20
mview = mode_viewer.ModeViewer(rod_list, modes[0:ndof, :], lambdas, amplitude=amplitude, width=1024, height=640)
mview.show()

AttributeError: 'Renderer' object has no attribute 'pauseRendering'

Select **Mode 7** to visualize the first mode associated to a non-zero eigenvalue

----
<a id='sec:constant_link'></a>
# Compute the equilibium state of a knotted elastic rod with **prescribed Link**

In [44]:
import vis, matplotlib
from linkage_vis import CenterlineMesh

def update_material_frame(viewer, pr):
    dc = pr.rod.deformedConfiguration()
    frameArray = np.array([d.d2 for d in dc.materialFrame])
    vmin, vmax = 0, 1.4  # yellow frame
    frame = vis.fields.VectorField(viewer.mesh, frameArray, vmin=vmin, vmax=vmax, colormap=matplotlib.cm.jet, glyph=vis.fields.VectorGlyph.CYLINDER)
    viewer.update(mesh=CenterlineMesh(pr.rod), vectorField=frame)
    

def compute_calugareanu_quantities(pr):
    """
    Compute quantities appearing in Calugareanu's theorem:
    Lk + \Phi / 2\pi = Tw + Wr
    """
    Lk = pr.link()
    Phi = pr.openingAngle()   # in [0, 2pi]
    Tw = pr.twist()
    Wr = pr.writhe()
    
    return Lk, Phi, Tw, Wr


def callback(problem, iteration):
    if iteration % 5 == 0:
        update_material_frame(view_framed_mat, rod_list[0])
        view_framed.update()

In [50]:
# Generate knot
p, q, a, r = 2, 3, 2, 4
rod_radius = 0.2

material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = generate_curve(torus_knot(p=p, q=q, a=a, r=r), npts=202)
pr = define_periodic_rod(centerline, material)
dc = pr.rod.deformedConfiguration()
rod_list = elastic_knots.PeriodicRodList([pr])

In [51]:
view_framed = Viewer(pr.rod, width=1024, height=640)
view_framed_mat = CenterlineViewer(pr.rod, superView=view_framed)

update_material_frame(view_framed_mat, pr)
view_framed.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), …

The yellow material frame represents the orientation of the cross-section

In [47]:
# Set the (generalized) linking number of the framed curve Lk + \Phi / 2\pi \in R^3
gen_link = -5.8

pr = set_generalized_link(pr, gen_link)
Lk_start, Phi_start, Tw_start, Wr_start = compute_calugareanu_quantities(pr)
rod_list = elastic_knots.PeriodicRodList(pr)

# Update the material frame in the viewer
update_material_frame(view_framed_mat, pr)
view_framed.update(mesh=rod_list)

In [48]:
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = [pr.thetaOffset(), pr.numDoF()-1]  # pin \theta_0 and \Theta (== totalOpeningAngle) to preserve the Link

In [49]:
# Optimize
report = elastic_knots.compute_equilibrium(
    rod_list, problemOptions, optimizerOptions, 
    fixedVars=fixedVars,
    externalForces=np.zeros(rod_list.numDoF()),
    softConstraints=[],
    callback=callback,
    hessianShift=hessianShift
)
view_framed.update()

0	4.76201	0.202036	0.202036	0.125	1
1	4.56574	0.594607	0.594607	0.5	0
2	4.51167	1.39069	1.39069	1	0
3	4.14587	0.700102	0.700102	0.5	1
4	4.13328	1.20711	1.20711	1	0
5	3.93795	0.244053	0.244053	0.25	1
6	3.9145	0.553647	0.553647	1	0
7	3.85758	0.17923	0.17923	0.25	0
8	3.84097	0.273067	0.273067	1	0
9	3.8137	0.359433	0.359433	1	0
10	3.78455	0.270394	0.270394	1	0
11	3.77541	0.832845	0.832845	1	0
12	3.7178	0.093535	0.093535	0.5	0
13	3.71038	0.871721	0.871721	1	0
14	3.68061	0.0596344	0.0596344	0.5	1
15	3.67103	0.355119	0.355119	1	1
16	3.6609	0.152643	0.152643	1	1
17	3.65522	0.351255	0.351255	1	0
18	3.64965	0.0634973	0.0634973	1	1
19	3.64748	0.117858	0.117858	1	0
20	3.64599	0.0818755	0.0818755	1	0
21	3.64538	0.0340328	0.0340328	1	0
22	3.64524	0.0196244	0.0196244	1	0
23	3.64522	0.00703165	0.00703165	1	1
24	3.64522	0.000420268	0.000420268	1	1
25	3.64521	6.90987e-05	6.90987e-05	1	1
26	3.64521	0.000358843	0.000358843	0.5	1
27	3.64521	0.000857571	0.000857571	0.0625	1
28	3.64521	0.00389659	0.00389659	

### Check Călugăreanu's Theorem

In [17]:
pr = rod_list[0]
Lk_end, Phi_end, Tw_end, Wr_end = compute_calugareanu_quantities(pr)

In [18]:
from prettytable import PrettyTable

RED, GREEN, END = '\033[91m', '\033[92m', '\033[0m'
rows = [
    ['Start', Lk_start, Phi_start/(2*np.pi), RED+str(Tw_start)+END, RED+str(Wr_start)+END, Lk_start + Phi_start/(2*np.pi), GREEN+str(Tw_start + Wr_start)+END], 
    ['End',   Lk_end,   Phi_end/(2*np.pi),   RED+str(Tw_end)+END,   RED+str(Wr_end)+END,   Lk_end + Phi_end/(2*np.pi),     GREEN+str(Tw_end + Wr_end)+END    ]
]
tab = PrettyTable(['', 'Link', 'Phi/2pi', 'Twist', 'Writhe', 'Link + Phi/2pi', 'Twist + Writhe'])
tab.float_format = '.3'
tab.add_rows(rows)
print(tab)

+-------+--------+---------+---------------------+--------------------+----------------+--------------------+
|       |  Link  | Phi/2pi |        Twist        |       Writhe       | Link + Phi/2pi |   Twist + Writhe   |
+-------+--------+---------+---------------------+--------------------+----------------+--------------------+
| Start | -6.000 |  0.200  | -2.2832738532379717 | -3.516726146762028 |     -5.800     |        -5.8        |
|  End  | -6.000 |  0.200  | -0.5850323105229517 | -5.214967689460405 |     -5.800     | -5.799999999983356 |
+-------+--------+---------+---------------------+--------------------+----------------+--------------------+


Twist gets traded for writhe during the optimization. Their sum stays constant.